In [129]:
from splinter import Browser
from bs4 import BeautifulSoup as bs
import time
import pymongo
import re
import pandas as pd
import requests

In [2]:
def init_browser():
    # @NOTE: Replace the path with your actual path to the chromedriver
    executable_path = {"executable_path": "/usr/local/bin/chromedriver"}
    return Browser("chrome", **executable_path, headless=False)

def soup_url (url):
    browser = init_browser()
    # Visit url
    browser.visit(url)
    time.sleep(1)

    # Scrape page into Soup
    html = browser.html
    soup_url = bs(html, "html.parser")
    
     # Close the browser after scraping
    browser.quit()
    
    return soup_url

def soup_class (soup, class_name):
    
    # Soup data list
    soup_class = soup.find(class_= class_name)
    
    return soup_class 

def soup_class_list (soup, class_name):
   
    # Soup data list
    soup_class_list = soup.find_all(class_= class_name)
    
    return soup_class_list

def text_list (soup_class_list):
    text_list = []
    [text_list.append(record.text.replace("\n", "")) for record in soup_class_list]
    return text_list

# WEB Scrapping Latest Oil News

In [3]:
# Visit oilprice.com
base_url = "https://oilprice.com"
url_news = "https://oilprice.com/Latest-Energy-News/World-News/"
soup = soup_url(url_news)

In [4]:
page_title = soup.title.text
page_title

'Oil News Today  | OilPrice.com'

In [33]:
image_holder = soup.find_all('img', src=True)
image_holder

[<img alt="Crude oil prices today - Oilprice.com" class="desktop_logo" src="https://d1o9e4un86hhpc.cloudfront.net/a/img/oilprice-logo.png?v=3" title="Oil prices - Oilprice.com"/>,
 <img alt="Crude oil prices today - Oilprice.com" class="mobile_logo" src="https://d1o9e4un86hhpc.cloudfront.net/a/img/common/header/mobile_logo.png?v=3" title="Oil prices - Oilprice.com"/>,
 <img alt="OilPrice GEA" id="main_nav_button" src="https://d1o9e4un86hhpc.cloudfront.net/a/img/common/nav_button_green.png" style="height: 25px;" title="Global Energy Alert"/>,
 <img alt="Link" height="21" src="https://d1o9e4un86hhpc.cloudfront.net/a/img/common/header/link.png" width="21"/>,
 <img alt="Link" src="https://d1o9e4un86hhpc.cloudfront.net/a/img/common/header/link.png"/>,
 <img alt="Link" src="https://d1o9e4un86hhpc.cloudfront.net/a/img/common/header/link.png"/>,
 <img alt="Link" height="21" src="https://d1o9e4un86hhpc.cloudfront.net/a/img/common/header/link.png" width="21"/>,
 <img alt="Link" height="21" src="

In [96]:
# Scrapping News Data
articles = soup.find_all(class_= "categoryArticle")
title_list = []
date_list = []
excerpt_list = []
author_list = []
link_list = []
for article in articles:
    image_holder = article.find(class_= "categoryArticle__imageHolder")
    img =image_holder.findAll('img')
    link = img[0].get('data-src')
    link_list.append(link)
    title = article.find_all(class_= "categoryArticle__title")
    full_date = article.find_all(class_= "categoryArticle__meta")
    date = full_date[0].text.strip().split("|")[0]
    author = full_date[0].text.strip().split("|")[1]
    excerpt = article.find_all(class_= "categoryArticle__excerpt")
    title_list.append(title[0].text.strip())
    date_list.append(date)
    author_list.append(author)
    excerpt_list.append(excerpt[0].text.strip())
link_list

['https://d32r1sh890xpii.cloudfront.net/news/270x200/2020-10-09_n75uqenxd4.jpg',
 'https://d32r1sh890xpii.cloudfront.net/news/270x200/2020-10-09_zsxvky4oan.jpg',
 'https://d32r1sh890xpii.cloudfront.net/news/270x200/2020-10-09_hsq3dpnwxu.jpg',
 'https://d32r1sh890xpii.cloudfront.net/news/270x200/2020-10-09_sf1ekjgrhv.jpg',
 'https://d32r1sh890xpii.cloudfront.net/news/270x200/2020-10-09_snvmtzmyui.jpg',
 'https://d32r1sh890xpii.cloudfront.net/news/270x200/2020-10-08_clwzrvfq4h.jpg',
 'https://d32r1sh890xpii.cloudfront.net/news/270x200/2020-10-08_pubc0h8o3e.jpg',
 'https://d32r1sh890xpii.cloudfront.net/news/270x200/2020-10-08_7cnn5jqtea.jpg',
 'https://d32r1sh890xpii.cloudfront.net/news/270x200/2020-10-08_guhajeatn4.jpg',
 'https://d32r1sh890xpii.cloudfront.net/news/270x200/2020-10-08_ysgobvf6dq.jpg',
 'https://d32r1sh890xpii.cloudfront.net/news/270x200/2020-10-08_lezwx3rmve.jpg',
 'https://d32r1sh890xpii.cloudfront.net/news/270x200/2020-10-08_drwn2xapps.jpg',
 'https://d32r1sh890xpii.clo

In [97]:
news = []
for i in range(len(title_list)):
    row = {}
    row['Date'] = date_list[i]
    row["News_Title"] = title_list[i]
    row["News_Paragraph"] = excerpt_list[i]
    row["Author"] = author_list[i]
    row["Image_URL"] = link_list[i]
    news.append(row)
news[:5]

[{'Date': 'Oct 09, 2020 at 13:29 ',
  'News_Title': 'Turkey’s Black Sea Gas Discovery May Be Bigger Than Thought',
  'News_Paragraph': 'Turkey is about to revise up the estimate of its natural gas discovery in the Black Sea, Bloomberg reported on Friday, quoting sources with direct knowledge of the plans.…',
  'Author': ' Tsvetana Paraskova',
  'Image_URL': 'https://d32r1sh890xpii.cloudfront.net/news/270x200/2020-10-09_n75uqenxd4.jpg'},
 {'Date': 'Oct 09, 2020 at 13:11 ',
  'News_Title': 'Norway’s Oil Strike Could End As Soon As Friday',
  'News_Paragraph': 'There is a chance that the oil workers’ strike in Norway will end as soon as on Friday, the head of the trade union whose members are on strike for…',
  'Author': ' Tsvetana Paraskova',
  'Image_URL': 'https://d32r1sh890xpii.cloudfront.net/news/270x200/2020-10-09_zsxvky4oan.jpg'},
 {'Date': 'Oct 09, 2020 at 12:48 ',
  'News_Title': 'Air Travel Collapse Makes Oil Hedges More Expensive',
  'News_Paragraph': 'The pandemic and the resu

# WEB Scrapping Latest Oil Prices

In [106]:
# Visit oilprice.com
url_prices = "https://oilprice.com/oil-price-charts/#prices"
soup_prices = soup_url(url_prices)

In [107]:
page_title = soup_prices.title.text
page_title

'Oil Price Charts | Oilprice.com'

In [131]:
price_holders = soup_prices.find_all(class_= "stripe show_graph update_on_load")
price_holders[:2]

[<tr class="stripe show_graph update_on_load" data-id="45" data-spreadsheet="Crude Oil WTI">
 <td class="flag_holder"><div class="flag_usa"></div></td><td>WTI Crude<i aria-hidden="true" class="fa fa-plus user_add_blend hidden" style="float:right;" title="Follow blend"></i><i aria-hidden="true" class="fa fa-times user_remove_blend hidden" style="float:right;" title="Unfollow blend"></i></td>
 <td class="last_price" data-price="40.6">40.60</td><td class="change_down flat_change_cell" dir="ltr">-0.59</td><td class="change_down_percent percent_change_cell" dir="ltr">-1.43%<span class="blend_update_text">(1 day Delay)</span></td><td class="last_updated" data-stamp="1602282600">(1 day Delay)</td></tr>,
 <tr class="stripe show_graph update_on_load" data-id="46" data-spreadsheet="Crude Oil Brent">
 <td class="flag_holder"><div class="flag_uk"></div></td><td>Brent Crude<i aria-hidden="true" class="fa fa-plus user_add_blend hidden" style="float:right;" title="Follow blend"></i><i aria-hidden="tr

In [161]:
r = requests.get(url_prices)
price_list = pd.read_html(r.text) # this parses all the tables in webpages to a list
price_df = price_list[0]
price_df

,0,1,2,3,4,5
0,NaN,WTI Crude,40.32,-0.28,-0.69%(11 Minutes Delay),(11 Minutes Delay)
1,NaN,Brent Crude,42.52,-0.33,-0.77%(11 Minutes Delay),(11 Minutes Delay)
2,NaN,Mars US,41.05,-0.89,-2.12%(40 Minutes Delay),(40 Minutes Delay)
3,NaN,Opec Basket,41.06,+0.61,+1.51%(3 \tdays Delay),(3 \tdays Delay)
4,NaN,Canadian Crude Index,29.73,-0.23,-0.77%(3 \tweeks Delay),(3 \tweeks Delay)
5,NaN,DME Oman,42.21,-0.76,-1.77%(2 \tdays Delay),(2 \tdays Delay)
6,NaN,Urals,39.60,+0.00,+0.00%(2 \tdays Delay),(2 \tdays Delay)
7,NaN,Mexican Basket,38.04,+1.13,+3.06%(3 \tdays Delay),(3 \tdays Delay)
8,NaN,Indian Basket,41.56,+0.48,+1.17%(3 \tdays Delay),(3 \tdays Delay)
9,NaN,Western Canadian Select,31.69,+1.49,+4.93%(2 \tdays Delay),(2 \tdays Delay)


In [157]:
prices = []
for price_row in price_df.iterrows():
    row = {}
    row['Oil_Symbol'] = price_row[1][1]
    row['Oil_Price'] = price_row[1][2]
    row['Oil_Change_Value'] = price_row[1][3]
    row['Oil_Change_Interest'] = price_row[1][4].split("(")[0]
    row['Oil_Price_Delay'] = price_row[1][5]
    prices.append(row)
prices[:10]

[{'Oil_Symbol': 'WTI Crude',
  'Oil_Price': '40.60',
  'Oil_Change_Value': '-0.59',
  'Oil_Change_Interest': '-1.43%',
  'Oil_Price_Delay': '(1 day Delay)'},
 {'Oil_Symbol': 'Brent Crude',
  'Oil_Price': '42.85',
  'Oil_Change_Value': '-0.49',
  'Oil_Change_Interest': '-1.13%',
  'Oil_Price_Delay': '(1 day Delay)'},
 {'Oil_Symbol': 'Mars US',
  'Oil_Price': '41.05',
  'Oil_Change_Value': '-0.89',
  'Oil_Change_Interest': '-2.12%',
  'Oil_Price_Delay': '(45 Minutes Delay)'},
 {'Oil_Symbol': 'Opec Basket',
  'Oil_Price': '41.06',
  'Oil_Change_Value': '+0.61',
  'Oil_Change_Interest': '+1.51%',
  'Oil_Price_Delay': '(3 \tdays Delay)'},
 {'Oil_Symbol': 'Canadian Crude Index',
  'Oil_Price': '29.73',
  'Oil_Change_Value': '-0.23',
  'Oil_Change_Interest': '-0.77%',
  'Oil_Price_Delay': '(3 \tweeks Delay)'},
 {'Oil_Symbol': 'DME Oman',
  'Oil_Price': '42.21',
  'Oil_Change_Value': '-0.76',
  'Oil_Change_Interest': '-1.77%',
  'Oil_Price_Delay': '(2 \tdays Delay)'},
 {'Oil_Symbol': 'Urals',


# Store Data in Database

In [98]:
# Initialize PyMongo to work with MongoDBs
conn = 'mongodb://localhost:27017'
client = pymongo.MongoClient(conn)

In [99]:
# Define database and collection
db = client.oil_db
collection = db.oil_news

In [100]:
# Upload collection
for record in news:
    collection.insert_one(record)

In [147]:
# Display the MongoDB records created above
oil_news_db = db.oil_news.find()
for record in oil_news_db[:10]:
    print(record)

{'_id': ObjectId('5f8247ff1b28d0739f5c7cf1'), 'Date': 'Oct 09, 2020 at 13:29 ', 'News_Title': 'Turkey’s Black Sea Gas Discovery May Be Bigger Than Thought', 'News_Paragraph': 'Turkey is about to revise up the estimate of its natural gas discovery in the Black Sea, Bloomberg reported on Friday, quoting sources with direct knowledge of the plans.…', 'Author': ' Tsvetana Paraskova', 'Image_URL': 'https://d32r1sh890xpii.cloudfront.net/news/270x200/2020-10-09_n75uqenxd4.jpg'}
{'_id': ObjectId('5f8247ff1b28d0739f5c7cf2'), 'Date': 'Oct 09, 2020 at 13:11 ', 'News_Title': 'Norway’s Oil Strike Could End As Soon As Friday', 'News_Paragraph': 'There is a chance that the oil workers’ strike in Norway will end as soon as on Friday, the head of the trade union whose members are on strike for…', 'Author': ' Tsvetana Paraskova', 'Image_URL': 'https://d32r1sh890xpii.cloudfront.net/news/270x200/2020-10-09_zsxvky4oan.jpg'}
{'_id': ObjectId('5f8247ff1b28d0739f5c7cf3'), 'Date': 'Oct 09, 2020 at 12:48 ', 'Ne

In [148]:
# Define new collection
prices_collection = db.oil_prices

In [149]:
# Upload collection
for record in prices:
    prices_collection.insert_one(record)

In [151]:
# Display the MongoDB records created above
oil_prices_db = db.oil_prices.find()
for record in oil_prices_db[:10]:
    print(record)

{'_id': ObjectId('5f8383741b28d0739f5c7d05'), 'Oil_Symbol': 'WTI Crude', 'Oil_Price': '40.60', 'Oil_Change_Value': '-0.59', 'Oil_Change_Interest': '-1.43%', 'Oil_Price_Delay': '(1 day Delay)'}
{'_id': ObjectId('5f8383751b28d0739f5c7d06'), 'Oil_Symbol': 'Brent Crude', 'Oil_Price': '42.85', 'Oil_Change_Value': '-0.49', 'Oil_Change_Interest': '-1.13%', 'Oil_Price_Delay': '(1 day Delay)'}
{'_id': ObjectId('5f8383751b28d0739f5c7d07'), 'Oil_Symbol': 'Mars US', 'Oil_Price': '41.05', 'Oil_Change_Value': '-0.89', 'Oil_Change_Interest': '-2.12%', 'Oil_Price_Delay': '(45 Minutes Delay)'}
{'_id': ObjectId('5f8383751b28d0739f5c7d08'), 'Oil_Symbol': 'Opec Basket', 'Oil_Price': '41.06', 'Oil_Change_Value': '+0.61', 'Oil_Change_Interest': '+1.51%', 'Oil_Price_Delay': '(3 \tdays Delay)'}
{'_id': ObjectId('5f8383751b28d0739f5c7d09'), 'Oil_Symbol': 'Canadian Crude Index', 'Oil_Price': '29.73', 'Oil_Change_Value': '-0.23', 'Oil_Change_Interest': '-0.77%', 'Oil_Price_Delay': '(3 \tweeks Delay)'}
{'_id': Ob

In [ ]:
client.close()